# Gradually increase the n_sats 
> Work in progress

In [22]:
from modules.golomb_problem import init_simple_problem, orbital_golomb_array
# --- --- --- --- ---
from IPython.display import display, Markdown, clear_output
from tqdm import tqdm

def show_table_of_solutions(fitness_and_solutions_list : list[tuple[float,list[float]]])-> None:
    table_header = "| N | Fitness | Solution      |\n|---|---|---|\n"
    for fitness_value, solution in fitness_and_solutions_list:
        x = []
        for x_i in solution:
            x.append(round(x_i, 2))   # Approssimazione alla terza cifra decimale
        table_header += f"| {len(solution)/6} | {fitness_value:.5} | {x} |\n"

    display(Markdown(table_header))

def increase_difficulty(udp : orbital_golomb_array, n_sats_range : range, optimizer) -> list[tuple[float, list[float]]]:
    result = []
    for n_sats in tqdm(n_sats_range, "Optimization on") :
        show_table_of_solutions(result)
        display(Markdown("---"))
        fitness, solution = optimizer(udp, n_sats)

        result.append((fitness, solution))
        clear_output()
    show_table_of_solutions(result)
    return result
# --- --- --- --- ---

UDP = init_simple_problem()

##### Template

In [23]:
import time
import random

udp = UDP,
n_sats_range = range(5,40,4)

def optimizer(udp : orbital_golomb_array, sats : int) -> tuple[float, list[float]] :
    print("among goose")
    time.sleep(2)
    
    return (random.random(), [random.random() for i in range(0,sats*6)])

ret = increase_difficulty(udp,n_sats_range,optimizer)
del ret

Optimization on: 100%|██████████| 9/9 [00:18<00:00,  2.02s/it]


| N | Fitness | Solution      |
|---|---|---|
| 5.0 | 0.74928 | [0.35, 0.29, 0.19, 0.53, 0.8, 0.53, 0.26, 0.8, 0.83, 0.18, 0.6, 0.53, 0.7, 0.23, 0.31, 0.28, 0.07, 0.8, 0.98, 0.3, 0.53, 0.83, 0.47, 0.71, 0.47, 0.58, 0.15, 0.35, 0.34, 0.45] |
| 9.0 | 0.36644 | [0.72, 0.06, 0.12, 0.54, 0.92, 0.94, 0.52, 0.33, 0.85, 0.08, 0.59, 0.2, 0.9, 0.75, 0.89, 0.17, 0.72, 0.9, 0.52, 0.59, 0.36, 0.58, 0.22, 0.51, 0.02, 0.51, 0.18, 0.09, 0.73, 0.19, 0.56, 0.8, 0.16, 0.44, 0.51, 1.0, 0.12, 0.03, 0.13, 0.9, 0.04, 0.54, 0.17, 0.6, 0.71, 0.18, 0.85, 0.27, 0.84, 0.55, 0.66, 0.74, 0.92, 0.97] |
| 13.0 | 0.4604 | [0.64, 0.44, 0.08, 0.17, 0.3, 0.81, 0.03, 0.56, 0.94, 0.13, 0.23, 0.74, 0.01, 0.21, 0.9, 0.75, 0.84, 0.3, 0.83, 0.05, 0.14, 0.68, 0.64, 0.51, 0.29, 0.64, 0.07, 0.74, 0.99, 0.99, 0.67, 0.13, 0.96, 0.67, 0.35, 0.25, 0.32, 0.61, 0.56, 0.67, 0.79, 0.61, 0.55, 0.77, 0.77, 0.97, 0.72, 0.88, 0.95, 0.21, 0.89, 0.28, 0.59, 0.03, 0.62, 0.49, 0.27, 0.27, 0.54, 0.86, 0.51, 0.84, 0.04, 0.67, 0.74, 0.26, 0.77, 0.17, 0.91, 0.88, 0.34, 0.7, 0.22, 0.52, 0.16, 0.76, 0.94, 0.68] |
| 17.0 | 0.39565 | [0.08, 0.95, 0.28, 0.51, 0.69, 0.47, 0.96, 0.94, 0.99, 0.49, 0.97, 0.06, 0.92, 0.04, 0.6, 0.84, 0.02, 0.65, 0.21, 0.26, 0.3, 0.29, 0.02, 0.5, 0.4, 0.54, 0.9, 0.26, 0.3, 0.75, 0.59, 0.6, 0.75, 0.27, 0.87, 0.62, 0.68, 0.25, 0.15, 0.38, 0.47, 0.93, 0.91, 0.56, 0.7, 0.11, 0.66, 0.67, 0.69, 0.16, 0.15, 0.47, 0.65, 0.74, 0.31, 0.76, 0.68, 0.71, 0.1, 0.83, 0.16, 0.17, 0.35, 0.15, 0.87, 0.94, 0.39, 0.13, 0.43, 0.72, 0.81, 0.09, 0.07, 0.41, 0.13, 0.22, 0.54, 0.47, 0.7, 0.26, 0.92, 0.7, 0.36, 0.06, 0.56, 0.95, 0.11, 0.49, 0.22, 0.32, 0.67, 0.47, 0.63, 0.58, 0.84, 0.37, 0.6, 0.6, 0.4, 0.26, 0.49, 0.02] |
| 21.0 | 0.14495 | [0.07, 0.6, 0.18, 0.87, 0.19, 0.75, 0.77, 0.09, 0.55, 0.86, 0.94, 0.69, 0.58, 0.71, 0.85, 0.09, 0.36, 0.75, 0.9, 0.9, 0.22, 0.6, 0.56, 0.05, 0.83, 0.67, 0.65, 0.16, 0.88, 0.98, 0.0, 0.21, 0.1, 0.98, 0.78, 0.33, 0.16, 0.34, 0.35, 0.06, 0.01, 0.18, 0.82, 0.32, 0.65, 0.24, 0.01, 0.42, 0.19, 0.01, 0.2, 0.58, 0.5, 0.7, 0.86, 0.41, 0.57, 0.77, 0.58, 0.72, 0.06, 0.64, 0.91, 0.37, 0.73, 0.12, 0.96, 0.32, 0.3, 0.66, 0.73, 0.55, 0.03, 0.45, 0.22, 0.11, 0.9, 0.34, 0.9, 0.47, 0.1, 0.86, 0.62, 0.78, 0.87, 0.48, 0.94, 0.23, 0.53, 0.99, 0.18, 0.18, 0.63, 0.37, 0.74, 0.67, 0.67, 0.89, 0.13, 0.55, 0.15, 0.51, 0.59, 0.99, 0.52, 0.84, 0.7, 0.04, 0.51, 0.12, 0.38, 0.55, 0.19, 0.12, 0.94, 0.53, 0.79, 0.87, 0.31, 0.99, 0.6, 0.29, 0.64, 0.65, 0.67, 0.02] |
| 25.0 | 0.45626 | [0.96, 0.21, 0.33, 0.35, 0.58, 0.68, 0.78, 0.49, 0.73, 0.06, 0.09, 0.02, 0.88, 0.94, 0.95, 0.15, 0.28, 0.72, 0.87, 0.23, 0.66, 0.53, 0.36, 0.42, 0.44, 0.66, 0.68, 0.92, 0.55, 0.02, 0.7, 0.29, 0.86, 0.9, 0.42, 0.24, 0.23, 0.24, 0.24, 0.23, 0.89, 0.97, 0.12, 0.9, 0.33, 0.55, 0.6, 0.6, 0.52, 0.95, 0.53, 0.54, 0.23, 0.92, 0.16, 0.41, 0.54, 0.52, 0.83, 0.95, 0.93, 0.39, 0.21, 0.1, 0.64, 0.08, 0.34, 0.04, 0.44, 0.18, 0.18, 0.18, 0.19, 0.06, 0.31, 0.05, 0.52, 0.05, 0.25, 0.04, 0.22, 0.31, 0.48, 0.83, 0.99, 0.1, 0.39, 0.58, 0.48, 0.16, 0.58, 0.82, 0.49, 0.8, 0.94, 0.43, 0.74, 0.23, 0.74, 0.57, 0.99, 0.72, 0.45, 0.51, 0.55, 0.99, 0.7, 0.99, 0.24, 0.95, 0.41, 0.96, 0.22, 0.83, 0.86, 0.4, 0.9, 0.34, 0.96, 0.6, 0.23, 0.13, 0.7, 0.39, 0.06, 0.08, 1.0, 0.65, 0.01, 0.79, 0.23, 0.24, 0.84, 0.85, 0.32, 0.15, 0.35, 0.21, 0.99, 0.06, 0.63, 0.19, 0.76, 0.75, 0.59, 0.21, 0.52, 0.44, 0.7, 0.27] |
| 29.0 | 0.15465 | [0.27, 0.67, 0.91, 0.47, 0.46, 0.84, 0.83, 0.85, 0.27, 0.8, 0.84, 0.72, 0.76, 0.42, 0.39, 0.24, 0.75, 0.04, 0.69, 0.85, 0.54, 0.85, 0.84, 0.07, 0.8, 0.32, 0.06, 0.99, 0.76, 0.05, 0.85, 0.71, 0.71, 0.2, 0.4, 0.22, 0.43, 0.84, 0.18, 0.02, 0.39, 0.61, 0.29, 0.7, 0.5, 0.63, 0.4, 0.95, 0.59, 0.48, 0.84, 0.73, 0.98, 0.21, 0.93, 0.04, 0.52, 0.56, 0.01, 0.91, 0.2, 0.65, 0.18, 0.62, 0.28, 0.36, 0.76, 0.1, 0.4, 0.2, 0.68, 0.18, 0.91, 0.74, 0.96, 0.41, 0.06, 0.02, 0.39, 0.88, 0.27, 0.94, 0.09, 0.63, 0.57, 0.94, 0.44, 0.97, 0.84, 0.5, 0.16, 0.79, 0.38, 0.21, 0.19, 0.17, 0.5, 0.79, 0.87, 0.34, 0.59, 0.75, 0.92, 0.67, 0.34, 0.6, 0.37, 0.98, 0.64, 0.36, 0.87, 0.93, 0.46, 0.7, 0.59, 0.29, 0.8, 0.17, 0.19, 0.37, 0.36, 0.35, 0.71, 0.11, 0.84, 0.44, 0.3, 0.88, 0.02, 0.93, 0.64, 0.3, 0.13, 0.96, 0.85, 0.09, 0.73, 0.57, 0.55, 0.32, 0.47, 0.19, 0.97, 0.47, 0.66, 0.59, 0.03, 0.92, 0.24, 0.09, 0.08, 0.41, 0.27, 0.87, 0.75, 0.59, 0.17, 0.63, 0.15, 0.52, 0.15, 0.51, 0.54, 0.44, 0.18, 0.49, 0.45, 0.22, 0.09, 0.81, 0.03, 0.82, 0.83, 0.7] |
| 33.0 | 0.00028573 | [0.8, 0.69, 0.14, 0.75, 0.98, 0.76, 0.06, 0.05, 0.1, 0.43, 0.04, 0.81, 0.31, 0.27, 0.91, 0.26, 0.74, 0.54, 0.45, 0.35, 0.83, 0.38, 0.1, 0.65, 0.41, 0.27, 0.17, 0.23, 0.53, 0.51, 0.11, 0.92, 0.62, 0.35, 1.0, 0.55, 0.52, 0.8, 0.79, 1.0, 0.07, 0.5, 0.9, 0.78, 0.08, 0.16, 0.67, 0.49, 0.53, 0.86, 0.79, 0.34, 0.01, 0.46, 0.49, 0.62, 0.03, 0.94, 0.08, 0.74, 0.16, 0.42, 0.78, 0.92, 0.46, 0.71, 0.33, 0.17, 0.81, 0.67, 0.8, 0.44, 0.26, 0.05, 0.04, 0.01, 0.94, 0.19, 0.57, 0.65, 0.38, 0.36, 0.2, 0.18, 0.27, 0.5, 0.25, 0.61, 0.28, 0.92, 0.89, 0.72, 0.33, 0.19, 0.08, 0.12, 0.98, 0.2, 0.53, 0.16, 0.15, 0.58, 0.74, 0.89, 0.9, 0.33, 0.51, 0.66, 0.02, 0.89, 0.45, 0.82, 0.79, 0.16, 0.71, 0.43, 0.94, 0.47, 0.73, 0.49, 0.07, 0.85, 0.62, 0.99, 0.95, 0.06, 0.24, 0.51, 0.07, 0.52, 0.95, 0.87, 0.26, 0.34, 0.02, 0.09, 0.07, 0.3, 0.63, 0.37, 0.72, 0.04, 0.69, 0.95, 0.03, 0.15, 0.0, 0.16, 0.38, 0.41, 0.25, 1.0, 0.23, 0.3, 0.51, 0.94, 0.63, 0.61, 0.94, 0.42, 0.95, 0.08, 0.2, 0.72, 0.28, 0.15, 0.69, 0.33, 0.04, 0.7, 0.86, 0.31, 0.76, 0.78, 0.03, 0.65, 0.84, 0.94, 0.75, 0.87, 0.65, 0.84, 0.97, 0.96, 0.92, 0.93, 0.68, 0.3, 0.84, 0.3, 0.43, 0.69, 0.95, 0.41, 0.35, 0.35, 0.38, 0.56] |
| 37.0 | 0.018171 | [0.53, 0.91, 0.29, 0.39, 0.7, 0.37, 0.35, 0.92, 0.25, 0.53, 0.79, 0.27, 0.39, 0.45, 0.21, 0.1, 0.29, 0.53, 0.62, 0.92, 0.05, 0.14, 0.03, 0.57, 0.74, 0.78, 0.38, 0.42, 0.32, 0.18, 0.52, 0.51, 0.14, 0.34, 0.24, 0.83, 0.48, 0.83, 0.89, 0.43, 0.88, 0.8, 0.42, 0.91, 0.91, 0.77, 0.58, 0.25, 0.88, 0.73, 0.12, 0.54, 0.84, 0.87, 0.19, 0.55, 0.66, 0.18, 0.26, 0.15, 0.27, 0.63, 0.88, 0.05, 0.53, 0.89, 0.7, 0.16, 0.44, 0.66, 0.87, 0.02, 0.61, 0.01, 0.87, 0.23, 0.63, 0.92, 0.44, 0.73, 0.2, 0.45, 0.13, 0.88, 0.97, 0.73, 0.02, 0.6, 0.11, 0.46, 0.5, 0.16, 0.34, 0.71, 0.84, 0.37, 0.06, 0.23, 0.43, 0.3, 0.34, 0.29, 0.23, 0.99, 0.82, 0.11, 0.76, 0.89, 0.62, 0.74, 0.23, 0.98, 0.87, 0.41, 0.74, 0.8, 0.81, 0.39, 0.49, 0.09, 0.97, 0.76, 0.67, 0.1, 0.76, 0.04, 0.11, 0.08, 0.41, 0.31, 0.53, 0.98, 0.69, 0.32, 0.65, 0.64, 0.05, 0.13, 0.88, 0.93, 0.45, 0.31, 0.65, 0.93, 0.89, 0.34, 0.73, 0.18, 0.1, 0.13, 0.17, 0.53, 0.67, 0.36, 0.33, 0.13, 0.7, 0.36, 0.52, 0.53, 0.8, 0.49, 0.59, 0.45, 0.38, 0.01, 0.7, 0.93, 0.27, 0.37, 0.35, 0.27, 0.12, 0.4, 0.73, 0.21, 0.91, 0.18, 0.43, 0.76, 0.85, 0.17, 0.46, 0.64, 0.29, 0.61, 0.92, 0.83, 0.95, 0.13, 0.14, 0.89, 0.32, 0.01, 0.8, 0.7, 0.83, 0.71, 0.6, 0.44, 0.57, 0.44, 0.37, 0.65, 0.78, 0.06, 0.61, 0.58, 0.46, 0.7, 0.34, 0.02, 0.24, 0.17, 0.44, 0.88, 0.97, 0.82, 0.29, 0.66, 0.9, 0.46] |


## Non-Dominated Sorting Genetic Algorithm

## Grey Wolf

## Differencial Evolution